In [139]:
import numpy as np
import pandas as pd
import math
df=pd.read_csv('projdata.csv',encoding = "utf-8",engine='python',sep=",")


In [140]:
y=np.array(df[['Fault']])
X=df.as_matrix(columns=df.columns[1:])
X=X.T
y=y.T

C:\Users\GOGULAANAND\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [141]:
def sig(x):
    return 1/(1+np.exp(-x))

In [142]:
nx=np.min(np.shape(X))
ny=np.min(np.shape(y))
print(X.shape)
nh=3
nl=1
print("Size of input layer: "+str(nx))
print("Size of output layer: "+str(ny))
print("Size of hidden units in each layer: "+str(nh))
print("No of hidden layers: "+str(nl))


(3, 3287)
Size of input layer: 3
Size of output layer: 1
Size of hidden units in each layer: 3
No of hidden layers: 1


In [143]:
def initialise_params(nx,nh,ny):
    
    W1=np.random.rand(nh,nx)*0.01
    b1=np.zeros((nh,1))
    W2=np.random.rand(ny,nh)*0.01
    b2=np.zeros((ny,1))
    
    assert (W1.shape == (nh, nx))
    assert (b1.shape == (nh, 1))
    assert (W2.shape == (ny, nh))
    assert (b2.shape == (ny, 1))
    
    params={"W1":W1,"b1":b1,"W2":W2,"b2":b2}
    
    return params
    

In [144]:
params=initialise_params(nx,nh,ny)

In [145]:
def forward_prop(X,params):
    
    W1=params["W1"]
    b1=params["b1"]
    W2=params["W2"]
    b2=params["b2"]
    
    Z1=np.dot(W1,X)
    A1=np.tanh(Z1)
    Z2=np.dot(W2,A1)
    A2=sig(Z2)
    
    cache={"Z1":Z1,"A1":A1,"Z2":Z2,"A2":A2}
    
    return A2,cache

In [146]:
A2,cache=forward_prop(X,params)

In [147]:
def compute_cost(y,A2):
    
    m=np.max(y.shape)
        
    c1=np.dot(y,(np.log(A2)).T)
    c2=np.dot((1-y),(np.log(1-A2)).T)
    cost = -((c1+c2)/m)
    
    cost = np.squeeze(cost) 
    return (np.sum(cost))

In [148]:
cost=compute_cost(y,A2)
print(cost)

0.6931567926220462


In [149]:
def back_prop(y,params,cache,X):
    
    W1=params["W1"]
    b1=params["b1"]
    W2=params["W2"]
    b2=params["b2"]
    
    m=np.max(y.shape)
    
    A1=cache["A1"]
    A2=cache["A2"]
    
    dZ2 = A2-y
    dW2 = np.dot(dZ2,A1.T)/m
    db2 = np.sum(dZ2,axis=1,keepdims=True)/m
    dZ1 = np.dot(W2.T,dZ2)*(1-np.power(A1,2))
    dW1 = np.dot(dZ1,X.T)/m
    db1 = np.sum(dZ1,axis=1,keepdims=True)/m
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads


In [150]:
grads=back_prop(y,params,cache,X)

In [151]:
def update_params(grads,params,learning_rate=1.2):
    
    W1 = params["W1"]
    b1 = params["b1"]
    W2 = params["W2"]
    b2 = params["b2"]
    
    dW1 = grads["dW1"]
    db1 = grads["db1"]
    dW2 = grads["dW2"]
    db2 = grads["db2"]
    
    W1 = W1-(learning_rate*dW1)
    b1 = b1-(learning_rate*db1)
    W2 = W2-(learning_rate*dW2)
    b2 = b2-(learning_rate*db2)
    
    
    params = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return params
    


In [152]:
learning_rate=0.2
params=update_params(grads,params,learning_rate)

In [153]:
def nn_model(X,y,nh,num_of_iters=100000,print_cost=False):
    
    nx=np.min(X.shape)
    ny=np.min(y.shape)
    
    parameters=initialise_params(nx,nh,ny)
    
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    learning_rate=1.2
    
    for i in range(num_of_iters):
        
        A2,cache=forward_prop(X,parameters)
        
        cost=compute_cost(y,A2)
        
        grads=back_prop(y,parameters,cache,X)
        
        parameters=update_params(grads,parameters,learning_rate)
        
        if print_cost and i % 1000 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
        
        return parameters

In [154]:
params=nn_model(X,y,nh,num_of_iters=10000,print_cost=True)

Cost after iteration 0: 0.693145


In [155]:
def predict(X,params):
    
    A2, cache = forward_prop(X,params)
    predictions = (A2>0.5)
    
    return predictions


In [156]:
predictions = predict(X,params)
print("predictions mean = " + str(np.mean(predictions)))

print ('Accuracy: %d' % float((np.dot(y,predictions.T) + np.dot(1-y,1-predictions.T))/float(y.size)*100) + '%')

predictions mean = 0.4998478856099787
Accuracy: 50%


In [157]:
hidden_layer_sizes = [1, 2, 3, 4, 5, 20, 50]
for i, nh in enumerate(hidden_layer_sizes):
    parameters = nn_model(X, y, nh, num_of_iters = 5000)
    predictions = predict(X,parameters)
    accuracy = float((np.dot(y,predictions.T) + np.dot(1-y,1-predictions.T))/float(y.size)*100)
    print ("Accuracy for {} hidden units: {} %".format(nh, accuracy))

Accuracy for 1 hidden units: 50.47155460906602 %
Accuracy for 2 hidden units: 50.501977487070285 %
Accuracy for 3 hidden units: 50.19774870702769 %
Accuracy for 4 hidden units: 50.228171585031944 %
Accuracy for 5 hidden units: 49.345908122908426 %
Accuracy for 20 hidden units: 50.167325829023426 %
Accuracy for 50 hidden units: 50.2585944630362 %
